In [ ]:
#import libraries
from difflib import SequenceMatcher
import os, os.path
import pandas as pd
import numpy as np
import pathlib
import shutil
import pysrt
import re
from num2fawords import words

In [ ]:
#check out similarity
def similar(a, b):
    cleana = re.findall(r'[ئابپتثجچحخآدذرزسشصضطظعغفقکگلمنوهی]+' , str(a))
    a = ''.join (cleana)
    cleanb = re.findall(r'[ئابپتثجچحخآدذرزسشصضطظعغفقکگلمنوهی]+' , str(b))
    b = ''.join (cleanb)
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
#parse subtitles 
def parser(path):
    sub = pysrt.open(path)
    return sub

In [ ]:
#sentence match
def matching (sub1,sub2):
    align = []
    for i in range (0,len(sub1)):
        time1_s = sub1[i].start.seconds + sub1[i].start.minutes*60
        time1_e = sub1[i].end.seconds + sub1[i].end.minutes*60
        text1 = sub1[i].text
        tx1 = len(text1.split(' '))
        for j in range (0,len(sub2)):
            time2_s = sub2[j].start.seconds + sub2[j].start.minutes*60
            time2_e = sub2[j].end.seconds + sub2[j].end.minutes*60
            text2 = sub2[j].text
            tx2 = len(text2.split(' '))
            if time1_s == time2_s and time1_e == time2_e and similar(text1,text2)<0.70 and (max(tx1,tx2)% min(tx1,tx2))<5 and min(tx1,tx2) >4:
                text1 = re.sub("[\(\[].*?[\)\]]", "", text1)
                text1 = re.sub("[\u06F0-\u06F90-9]+", "<عدد>" , text1)
                text1 = re.sub("[^U+200C ‌اآبپتثجچحخدذرزژسشصضطظعغفق><کگلمنوهیئء‍‍‍‍؟]+", " ", text1)
                text2 = re.sub("[\(\[].*?[\)\]]", "", text2)
                text2 = re.sub("[\u06F0-\u06F90-9]+", "<عدد>" , text2)
                text2 = re.sub("[^U+200C ‌اآبپتثجچحخدذرزژسشصضطظعغفق><کگلمنوهیئء‍‍‍‍؟]+", " ", text2)
                align.append([text1, text2])
                break
            else:
                pass
    return align

In [ ]:
#addressing and findout locations in memory
parent_dir = "Path to your subtitle's Folder"
files = next(os.walk(parent_dir))
len(files[1])

In [ ]:
#main body
m =[]
for file in files[1]:
    print (r'{}'.format(file))
    os.chdir(parent_dir)
    path = os.path.join(parent_dir, file)
    fils = next(os.walk(path))
    fis = len(fils[2])
    for i in range (0,fis-1):
        k = i+1
        path1 = str (fils[0] +'/'+ fils[2][i])
        a = fils[2][i].replace('.srt','')
        sub1 = parser(path1)
        for j in range (k,fis):
            path2 = str (fils[0] +'/'+ fils[2][j])
            b = fils[2][j].replace('.srt','')
            if a in path2 or b in path1 :
                sub2 = parser(path2)
                m =[*m,*matching(sub1,sub2)]
            else:
                continue

In [ ]:
lst = pd.DataFrame(m)
lst

In [ ]:
# write to an excel(optional)
df = pd.DataFrame(m)
writer = pd.ExcelWriter('.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='test1', index=False)
writer.save()

In [ ]:
#write to a CSV file
lst.to_csv('your_name.csv',index=False)